# Big Data Processing: Pandas and PySpark

**What we'll cover:**
1. Small data → Pandas wins
2. Large data → PySpark required
3. Side-by-side: Same operations in both tools
4. Key concepts: Lazy evaluation, actions vs transformations, partitions
5. Joins: Combining datasets
6. Common beginner mistakes
7. Writing results
8. Your turn: Practice exercises

---

## Setup

In [0]:
import pandas as pd
import time
from pyspark.sql.functions import col, count, avg, sum as spark_sum, desc, when

# Verify Spark is ready (spark is pre-created in Databricks)
print(f"✅ Spark version: {spark.version}")
print(f"✅ Ready to go!")

---
## Part 1: Small Data (100 MB) - Pandas is Faster

**Dataset:** Flight delays  
**Size:** ~100 MB, 1.4M rows  
**Task:** Find top 10 airports with most delays

**Hypothesis:** PySpark overhead makes it slower for small data.

### Pandas Approach

In [0]:
# PANDAS: Read and process
path = "/dbfs/databricks-datasets/flights/departuredelays.csv"

start = time.time()

df_pandas = pd.read_csv(path)
result = df_pandas[df_pandas['delay'] > 0] \
    .groupby('origin')['delay'] \
    .count() \
    .sort_values(ascending=False) \
    .head(10)

pandas_time = time.time() - start

print(f"⏱️  Pandas: {pandas_time:.2f} seconds")
print(f"\n📊 Top 10 airports:")
print(result)

### PySpark Approach

In [0]:
# PYSPARK: Same task
path = "/databricks-datasets/flights/departuredelays.csv"

start = time.time()

df_spark = spark.read.csv(path, header=True, inferSchema=True)
result = df_spark.filter(col('delay') > 0) \
    .groupBy('origin') \
    .count() \
    .orderBy(desc('count')) \
    .limit(10)

result.show()

pyspark_time = time.time() - start

print(f"\n  PySpark: {pyspark_time:.2f} seconds")

In [0]:
# COMPARISON
print("="*50)
print("Small Data (100 MB)")
print("="*50)
print(f"Pandas:  {pandas_time:.2f}s  ← WINNER")
print(f"PySpark: {pyspark_time:.2f}s")
print(f"\nPySpark is SLOWER")
print("\n💡 Why? PySpark has 2-3 second overhead:")
print("   - Task scheduling")
print("   - Data serialization")
print("   - Cluster coordination")
print("\n   For small data, overhead > processing time!")

---
## Part 2: Large Data (1 GB+) - PySpark Required

**Dataset:** 10x duplicated flights  
**Size:** ~1 GB, 14M rows  
**Task:** Same analysis on bigger data

**What happens:** Pandas will be slow/crash. PySpark handles it easily.

In [0]:
# Create large dataset by duplicating
print("Creating large dataset (10x duplication)...")

df_base = spark.read.csv("/databricks-datasets/flights/departuredelays.csv", 
                          header=True, inferSchema=True)

df_large = df_base
for i in range(9):
    df_large = df_large.union(df_base)

print("Created ~1 GB dataset with 14M rows")

In [0]:
# PYSPARK: Process large data
start = time.time()

result_large = df_large.filter(col('delay') > 0) \
    .groupBy('origin') \
    .count() \
    .orderBy(desc('count')) \
    .limit(10)

result_large.show()

large_time = time.time() - start

print(f"\n⏱️  PySpark on 1 GB: {large_time:.2f} seconds")
print(f"\n💡 Pandas would:")
print(f"   - Take 3-5x longer ({large_time*4:.1f}s estimate)")
print(f"   - Or crash with MemoryError")
print(f"\n   PySpark handles it easily! ✅")

---
## Part 3: Side-by-Side Comparison

Let's see common operations in both tools.

### Operation 1: Select Columns

In [0]:
# PANDAS
df_pandas_select = df_pandas[['date', 'origin', 'destination', 'delay']]
print("Pandas:")
print(df_pandas_select.head())

print("\n" + "="*50 + "\n")

# PYSPARK  
df_spark_select = df_spark.select('date', 'origin', 'destination', 'delay')
print("PySpark:")
df_spark_select.show(5)

### Operation 2: Filter Rows

In [0]:
# PANDAS
df_pandas_filter = df_pandas[
    (df_pandas['delay'] > 60) & 
    (df_pandas['distance'] > 1000)
]
print(f"Pandas: {len(df_pandas_filter):,} flights with >1hr delay and >1000 miles")

print("\n" + "="*50 + "\n")

# PYSPARK
df_spark_filter = df_spark.filter(
    (col('delay') > 60) & 
    (col('distance') > 1000)
)
print(f"PySpark: {df_spark_filter.count():,} flights with >1hr delay and >1000 miles")

### Operation 3: Add New Column

In [0]:
# PANDAS: Modify dataframe directly
df_pandas_copy = df_pandas.copy()
df_pandas_copy['delay_hours'] = df_pandas_copy['delay'] / 60
print("Pandas:")
print(df_pandas_copy[['delay', 'delay_hours']].head())

print("\n" + "="*50 + "\n")

# PYSPARK: Returns new dataframe (immutable)
df_spark_with_hours = df_spark.withColumn('delay_hours', col('delay') / 60)
print("PySpark:")
df_spark_with_hours.select('delay', 'delay_hours').show(5)

### Operation 4: Group By & Aggregate

In [0]:
# PANDAS
pandas_agg = df_pandas.groupby('origin').agg({
    'delay': ['mean', 'count']
}).round(2).head()
print("Pandas:")
print(pandas_agg)

print("\n" + "="*50 + "\n")

# PYSPARK
spark_agg = df_spark.groupBy('origin').agg(
    avg('delay').alias('avg_delay'),
    count('*').alias('count')
)
print("PySpark:")
spark_agg.show(5)

### Operation 5: Sort

In [0]:
# PANDAS
pandas_sorted = df_pandas.sort_values('delay', ascending=False).head(5)
print("Pandas - Top 5 delays:")
print(pandas_sorted[['date', 'origin', 'destination', 'delay']])

print("\n" + "="*50 + "\n")

# PYSPARK
spark_sorted = df_spark.orderBy(desc('delay')).limit(5)
print("PySpark - Top 5 delays:")
spark_sorted.select('date', 'origin', 'destination', 'delay').show()

### Summary: Syntax Comparison

| Operation | Pandas | PySpark |
|-----------|--------|----------|
| **Select** | `df[['col1', 'col2']]` | `df.select('col1', 'col2')` |
| **Filter** | `df[df['col'] > 10]` | `df.filter(col('col') > 10)` |
| **Add column** | `df['new'] = df['old'] * 2` | `df.withColumn('new', col('old') * 2)` |
| **Group by** | `df.groupby('col').sum()` | `df.groupBy('col').sum()` |
| **Sort** | `df.sort_values('col')` | `df.orderBy('col')` |

💡 **Key difference:** PySpark uses `col()` function for columns

---
## Part 4: Key PySpark Concepts

Understanding these concepts is critical for working effectively with PySpark.

### Concept 1: Lazy Evaluation

**Pandas:** Executes immediately  
**PySpark:** Builds a plan, executes when needed

In [0]:

# Lazy Evaluation Example 

# Intentionally inefficient order:
# 1. Create column for ALL rows (expensive)
# 2. Filter afterwards (wasteful)

df = spark.read.csv("/databricks-datasets/flights/departuredelays.csv", 
                    header=True, inferSchema=True)

query = (df
    .select('origin', 'destination', 'delay', 'distance')
    .withColumn('delay_category',
                when(col('delay') > 100, 'severe')
                .when(col('delay') > 30, 'moderate')
                .otherwise('minor'))
    .filter(col('delay') > 60)           # Written AFTER withColumn
    .filter(col('distance') > 500)       
    .groupBy('origin')
    .agg(avg('delay').alias('avg_delay'), count('*').alias('count'))
    .orderBy(desc('avg_delay'))
)

display(query)

### Concept 2: Actions vs Transformations (CRITICAL!)

This is the most important concept in PySpark!

**Transformations** (lazy - just build the plan):
- `.filter()`, `.select()`, `.groupBy()`, `.join()`, `.withColumn()`
- Don't execute anything
- Return a new DataFrame

**Actions** (trigger execution):
- `.count()`, `.show()`, `.collect()`, `.write()`
- Actually run the computation
- Return results to driver

In [0]:
# Actions vs Transformations

import time

df = spark.read.csv("/databricks-datasets/flights/departuredelays.csv", 
                    header=True, inferSchema=True)

# Transformations - lazy (just build a plan)
start = time.time()
filtered = df.filter(col('delay') > 100)
selected = filtered.select('origin', 'delay', 'distance')
print(f"Transformations: {time.time() - start:.4f}s")

# Actions - eager (trigger execution)
print("\nAction 1:")
start = time.time()
count = selected.count()
print(f"count() = {count} rows, took {time.time() - start:.2f}s")

print("\nAction 2:")
start = time.time()
selected.show(5)
print(f"show() took {time.time() - start:.2f}s")

print("\nNotice: Each action re-executes the transformations!")

### Quick Reference: Actions vs Transformations

| Type | Operations | What They Do |
|------|-----------|-------------|
| **Transformations** | `.filter()`, `.select()`, `.groupBy()`, `.join()`, `.withColumn()`, `.orderBy()` | Build execution plan (lazy) |
| **Actions** | `.count()`, `.show()`, `.collect()`, `.take()`, `.first()`, `.write()` | Trigger execution |

**Rule of thumb:** If it returns a DataFrame, it's a transformation. If it returns a value or writes data, it's an action.

### Concept 3: Partitions - How Spark Parallelizes

Data is split into **partitions** - think of them as chunks that can be processed in parallel.

- More partitions = more parallelism = faster (usually)
- But too many = overhead from coordination
- Rule of thumb: 2-4 partitions per CPU core

In [0]:
print("📦 Partitions Demo\n")
print("💡 Partitions split your data into chunks for parallel processing")
print("   - Databricks serverless automatically manages partitions")
print("   - You can still manually repartition if needed\n")

# Repartition to different sizes
print("Testing different partition strategies...\n")

# Fewer partitions (4)
df_few = df_large.repartition(4)
start = time.time()
df_few.filter(col('delay') > 0).count()
few_time = time.time() - start
print(f"✓ Fewer partitions (4): {few_time:.2f}s")

# More partitions (32)
df_many = df_large.repartition(32)
start = time.time()
df_many.filter(col('delay') > 0).count()
many_time = time.time() - start
print(f"✓ More partitions (32): {many_time:.2f}s")

# Default (let Spark decide)
start = time.time()
df_large.filter(col('delay') > 0).count()
default_time = time.time() - start
print(f"✓ Default (auto): {default_time:.2f}s")

print(f"\n💡 Key takeaways:")
print(f"   - More partitions ≠ always faster")
print(f"   - Too few = not enough parallelism")
print(f"   - Too many = coordination overhead")
print(f"   - Usually best to let Spark decide!")
print(f"\n📝 Note: On standard clusters, you can check partition count with:")
print(f"   df.rdd.getNumPartitions() (not available on serverless)")

### Concept 4: Filter Early = Faster

Always filter data BEFORE expensive operations like groupBy or join.

In [0]:
print("🎯 Optimization: Filter Early\n")

# BAD: Group all 14M rows first
start = time.time()
bad = df_large.groupBy('origin').count().filter(col('count') > 1000).count()
bad_time = time.time() - start
print(f"Bad:  {bad_time:.2f}s (grouped all data)")

# GOOD: Filter to 7M rows first
start = time.time()
good = df_large.filter(col('delay') > 0).groupBy('origin').count().filter(col('count') > 1000).count()
good_time = time.time() - start
print(f"Good: {good_time:.2f}s (filtered first)")

print(f"\n⚡ {bad_time/good_time:.1f}x speedup by filtering early!")

---
## Part 5: Joins - Combining DataFrames

Joins are one of the most common operations in real-world data work.

### Create Sample Airport Information Dataset

In [0]:
# Create a small airport info dataset
airport_data = [
    ('SFO', 'San Francisco', 'California'),
    ('LAX', 'Los Angeles', 'California'),
    ('ORD', 'Chicago', 'Illinois'),
    ('ATL', 'Atlanta', 'Georgia'),
    ('DFW', 'Dallas', 'Texas'),
    ('JFK', 'New York', 'New York'),
    ('SEA', 'Seattle', 'Washington'),
    ('DEN', 'Denver', 'Colorado')
]

airport_df = spark.createDataFrame(airport_data, ['code', 'city', 'state'])
print("Airport Information:")
airport_df.show()

### Inner Join - Get Delays with Airport Info

In [0]:
print("🔗 INNER JOIN Example\n")

# Get top delayed flights with city names
delayed_flights = df_spark.filter(col('delay') > 60).select('origin', 'delay', 'destination')

# Join to get origin city name
result = delayed_flights.join(
    airport_df,
    delayed_flights.origin == airport_df.code,
    'inner'  # only keep matches
).select(
    col('origin'),
    col('city').alias('origin_city'),
    col('state'),
    col('destination'),
    col('delay')
).orderBy(desc('delay'))

print("Top delayed flights with city names:")
result.show(10)

print("\n💡 Inner join only keeps rows that match in BOTH DataFrames")

### Left Join - Keep All Flights, Add Info When Available

In [0]:
print("🔗 LEFT JOIN Example\n")

# Left join keeps all flights, adds city info when available
result_left = delayed_flights.join(
    airport_df,
    delayed_flights.origin == airport_df.code,
    'left'  # keep all from left (delayed_flights)
).select(
    col('origin'),
    col('city').alias('origin_city'),
    col('destination'),
    col('delay')
)

print("All flights (with city name when available):")
result_left.show(10)

# Count nulls (airports not in our lookup table)
null_count = result_left.filter(col('origin_city').isNull()).count()
print(f"\n💡 Left join kept all flights: {null_count:,} have no city info (NULL)")

### Join Types Quick Reference

| Join Type | Keeps | Use When |
|-----------|-------|----------|
| **inner** | Only matches | Need complete info for both sides |
| **left** | All from left + matches from right | Keep all primary data, enrich with lookup |
| **right** | All from right + matches from left | Rarely used (just flip to left) |
| **outer** | Everything from both | Need to see all data, matched or not |

---
## Part 6: Common Beginner Mistakes (AVOID THESE!)

These mistakes will crash your code or make it super slow.

### Mistake 1: Using .collect() on Large Data

**THE CARDINAL SIN OF PYSPARK**

In [0]:
print("MISTAKE #1: .collect() on large data\n")

# ❌ NEVER DO THIS on large data
print("❌ BAD: df_large.collect()")
print("   → Brings ALL data to driver memory")
print("   → WILL CRASH with OutOfMemory error")
print("   → Use .show() or .limit() instead\n")

# ✅ GOOD alternatives
print("✅ GOOD: Peek at data")
df_large.show(20)  # Safe - only shows 20 rows

print("\n✅ GOOD: Sample for local analysis")
sample = df_large.limit(1000).toPandas()  # Safe - limited to 1000 rows
print(f"Sample size: {len(sample)} rows")

print("\n💡 RULE: Never .collect() unless you're sure data fits in memory!")

### Mistake 2: Excessive .count() Calls

In [0]:
print("⚠️ MISTAKE #2: Too many .count() calls\n")

# ❌ BAD: Multiple counts
print("❌ BAD: Counting at every step")
start = time.time()

step1 = df_large.filter(col('delay') > 0)
print(f"   After filter: {step1.count():,}")  # Expensive!

step2 = step1.filter(col('distance') > 500)
print(f"   After filter 2: {step2.count():,}")  # Expensive!

bad_time = time.time() - start
print(f"   Time: {bad_time:.2f}s\n")

# ✅ GOOD: Count only at the end
print("✅ GOOD: Count once at the end")
start = time.time()

result = df_large \
    .filter(col('delay') > 0) \
    .filter(col('distance') > 500)

final_count = result.count()
good_time = time.time() - start

print(f"   Final count: {final_count:,}")
print(f"   Time: {good_time:.2f}s")
print(f"\n⚡ {bad_time/good_time:.1f}x faster!\n")

print("💡 RULE: .count() is expensive. Use it sparingly!")

### Mistake 3: Not Viewing Execution Plans

In [0]:
print("🔍 Use .explain() to see what Spark is doing\n")

query = df_large \
    .filter(col('delay') > 60) \
    .groupBy('origin') \
    .count() \
    .orderBy(desc('count'))

# See the execution plan
print("Execution plan:")
query.explain()

print("\n💡 .explain() helps you understand:")
print("   - What operations Spark will do")
print("   - In what order")
print("   - Where optimizations happen")
print("   - Useful for debugging slow queries!")

### Mistake 4: Forgetting to Filter Before Joins

In [0]:
print("🎯 MISTAKE #4: Not filtering before joins\n")

# ❌ BAD: Join all data
print("❌ BAD: Join everything first")
start = time.time()
bad_result = df_large \
    .join(airport_df, df_large.origin == airport_df.code, 'inner') \
    .filter(col('delay') > 100) \
    .count()
bad_time = time.time() - start
print(f"   Time: {bad_time:.2f}s\n")

# ✅ GOOD: Filter first
print("✅ GOOD: Filter before join")
start = time.time()
good_result = df_large \
    .filter(col('delay') > 100) \
    .join(airport_df, df_large.origin == airport_df.code, 'inner') \
    .count()
good_time = time.time() - start
print(f"   Time: {good_time:.2f}s")
print(f"\n⚡ {bad_time/good_time:.1f}x faster!\n")

print("💡 RULE: Always filter before expensive operations like joins!")

### Common Mistakes Summary

| ❌ Don't Do This | ✅ Do This Instead |
|-----------------|-------------------|
| `.collect()` on large data | `.show()` or `.limit().toPandas()` |
| `.count()` after every step | `.count()` only when needed |
| Never check execution plan | Use `.explain()` to understand |
| Join then filter | Filter then join |
| Large `.show(10000)` | `.show(20)` is usually enough |
| Ignore errors | Read error messages (they're helpful!) |

**Remember: PySpark is lazy. Take advantage of it!**

___________________

## Part 7: Running SQL Queries in PySpark

PySpark allows you to use SQL syntax alongside DataFrame operations. This is useful if you're more comfortable with SQL or need to work with existing SQL queries.

**Key steps:**
1. **Register a DataFrame as a temporary view** using `.createOrReplaceTempView()`
2. **Run SQL queries** using `spark.sql()`
3. **Mix and match** - You can combine SQL queries with DataFrame operations

The result of `spark.sql()` is a DataFrame, so you can chain additional transformations or actions on it.

In [0]:
# Running SQL queries in PySpark

df = spark.read.csv("/databricks-datasets/flights/departuredelays.csv", 
                    header=True, inferSchema=True)

# Register DataFrame as a temporary view
df.createOrReplaceTempView("flights")

# Now you can use SQL
result = spark.sql("""
    SELECT origin, 
           AVG(delay) as avg_delay,
           COUNT(*) as flight_count
    FROM flights
    WHERE delay > 100
    GROUP BY origin
    ORDER BY avg_delay DESC
    LIMIT 10
""")

result.show()

# Can also mix SQL and DataFrame operations
spark.sql("SELECT * FROM flights WHERE delay > 200").filter(col('distance') > 1000).show(5)

---
## Part 8: Writing Results

You've processed your data - now save it!

### Write to Parquet (Recommended for big data)

In [0]:
print("💾 Writing Results\n")

# Create a result to save
result_to_save = df_large \
    .filter(col('delay') > 60) \
    .select('date', 'origin', 'destination', 'delay', 'distance')
    
print(f"Preview of what would be saved: {result_to_save.count():,} rows")
result_to_save.show(5)

### Write to CSV (For small results or Excel)

In [0]:
small_result = (
    df_large
    .filter(
        col('delay') > 60
    )
    .groupBy('origin')
    .agg(
        avg('delay').alias('avg_delay'),
        count('*').alias('flight_count')
    )
    .orderBy(
        desc('avg_delay')
    )
    .limit(100)
)

display(small_result)

### Write Modes

| Mode | What It Does | Use When |
|------|-------------|----------|
| `overwrite` | Replace existing data | Recreating results |
| `append` | Add to existing data | Incremental updates |
| `ignore` | Skip if exists | Don't want to overwrite |
| `error` | Fail if exists | Safety - don't overwrite accidentally |

**Most common:** `overwrite` for one-off analyses

### File Formats Quick Reference

| Format | Best For | Pros | Cons |
|--------|----------|------|------|
| **Parquet** | Big data storage | Fast, compressed, columnar | Not human-readable |
| **CSV** | Small results, Excel | Readable, universal | Slow, no compression |
| **JSON** | Nested data, APIs | Flexible schema | Larger files |
| **Delta** | Production data lakes | ACID, time travel, fast | Needs Delta Lake setup |

**Default choice: Use Parquet for everything unless you have a reason not to.**